In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [ ]:
def select(long1, lat1, long2, lat2):
    rectangle = ee.Geometry.Rectangle([long1, lat1, long2, lat2])

    dataset = "LANDSAT/LC08/C01/T1_SR"

    l8 = ee.ImageCollection(dataset)

    img = ee.Image(
        l8.filterBounds(rectangle)
            .filterDate('2015-01-01',
                '2015-12-31')
            .sort('CLOUD_COVER').first()
    )
    return img.clip(rectangle).select(list(range(11)))

In [ ]:
!pip install spyndex

In [ ]:
import spyndex

In [ ]:
!pip install -q kaggle

In [ ]:
! mkdir ~/.kaggle

! cp /content/drive/MyDrive/important/kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d reymaster/hwsd-landsat-processed

In [ ]:
! unzip "hwsd-landsat-processed.zip" -d latdata

In [ ]:
!ls

In [ ]:
import pandas as pd
latdata = pd.read_csv('latdata/latdata/latdata.csv').to_numpy()
latdata.shape

In [ ]:
def compute_indices (img):
    dataset = "LANDSAT/LC08/C01/T1_SR"

    parameters = {
        "A": img.select("B1"),
        "B": img.select("B2"),
        "G": img.select("B3"),
        "R": img.select("B4"),
        "N": img.select("B5"),
        "S1": img.select("B6"),
        "S2": img.select("B7"),
        "T1": img.select("B10"),
        "T2": img.select("B11"),
        "L" : 1, 
        "g" : 2.5, "C1" : 6, "C2" : 7.5
    }


    indices = ['AFRI1600','AFRI2100','ANDWI','AVI','AWEInsh','AWEIsh','BAI','BAIM','BCC','BI','BLFEI','BNDVI','BRBA','BaI','CIG','CSI','CSIT','CVI','DBI','DBSI','DSI','DSWI1','DSWI2','DSWI3','DSWI4','DSWI5','DVI','EBBI','EMBI','EVI','EVI2','ExG','ExGR','ExR','FCVI','GARI','GBNDVI','GCC','GEMI','GLI','GNDVI','GOSAVI','GRNDVI','GRVI','GSAVI','GVMI','IBI','IKAW','IPVI','LSWI','MBI','MCARI1','MCARI2','MGRVI','MIRBI','MLSWI26','MLSWI27','MNDVI','MNDWI','MNLI','MRBVI','MSAVI','MSI','MSR','MTVI1','MTVI2','MuWIR','NBAI','NBLI','NBR','NBR2','NBRSWIR','NBRT1','NBRT2','NBRT3','NBSIMS','NBUI','NDBI','NDBaI','NDDI','NDGlaI','NDII','NDISIb','NDISIg','NDISImndwi','NDISIndwi','NDISIr','NDMI','NDPonI','NDSI','NDSII','NDSWIR','NDSaII','NDTI','NDVI','NDVIMNDWI','NDVIT','NDWI','NDYI','NGRDI','NIRv','NLI','NMDI','NRFIg','NRFIr','NSDS','NSDSI1','NSDSI2','NSDSI3','NSTv1','NSTv2','NWI','NormG','NormNIR','NormR','OSAVI','PISI','RCC','RDVI','RGBVI','RGRI','RI','RI4XS','S3','SARVI','SAVI','SAVIT','SI','SIPI','SR','SR2','SWI','SWM','TDVI','TGI','TVI','TriVI','UI','VARI','VI6T','VIBI','VIG','VgNIRBI','VrNIRBI','WI1','WI2','WI2015','WRI']
    

    for x in range(0, len(indices), 2):
        try:
            img = img.addBands(spyndex.computeIndex([indices[x], indices[x + 1]], parameters))
        except Exception as e:
            pass

    meanDict = img.reduceRegion(
        reducer= ee.Reducer.mean(),
        geometry= img.geometry(),
        scale= 90,
        maxPixels= 40e9
    )

    # mappedDictionary = meanDict.map(lambda key, val:  val if val is not None else 0)

    return meanDict


In [ ]:
import numpy as np
sample = latdata[np.random.randint(latdata.shape[0], size=10000), :]

In [ ]:
def process_image(long1, lat1, long2, lat2, MU_GLOBAL):
	# Combine selecting area, pan-sharpening, illumination, topographic correction, and spectral indices

    img = select(long1, lat1, long2, lat2)

    img = img.scaleAndOffset()    
    indices = compute_indices(img);

    return indices.set('MU_GLOBAL', MU_GLOBAL).getInfo()

features = [];

for i in range(len(sample)):
    processed = process_image(sample[i][1], sample[i][3], sample[i][2], sample[i][4], sample[i][0]);
    features.append(processed);
    print(i)
    if i % 100 == 0:
        df =  pd.DataFrame(features)
        df.to_csv("/content/drive/MyDrive/SOC/Data/SRFastFeatures.csv", index = False)